In [1]:
import jax
import jax.numpy as jnp
import mujoco
import numpy as np
from mujoco import mjx
from robot_descriptions.z1_mj_description import MJCF_PATH

key = jax.random.PRNGKey(0)

mjmodel = mujoco.MjModel.from_xml_path(MJCF_PATH)
mjdata = mujoco.MjData(mjmodel)

# alter the model so it becomes mjx compatible
mjmodel.dof_frictionloss = 0
mjmodel.opt.integrator = 0

mjxmodel = mjx.put_model(mjmodel)
mjxdata = mjx.put_data(mjmodel, mjdata)

2024-07-03 16:08:21.341305: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
/home/lvjonok/github.com/lvjonok/mujoco-sysid/venv/lib/python3.10/site-packages/jax/_src/interpreters/xla.py:155: RuntimeWarning: overflow encountered in cast
  return np.asarray(x, dtypes.canonicalize_dtype(x.dtype))


In [2]:
MJCF_PATH

'/home/lvjonok/.cache/robot_descriptions/mujoco_menagerie/unitree_z1/z1.xml'

In [3]:
q, v, dv = jax.random.normal(key, (3, mjmodel.nq))

q, v, dv

(Array([-0.87181777, -0.13588488, -1.765594  ,  0.22557412, -1.3263583 ,
        -0.6441258 ], dtype=float32),
 Array([ 0.95800805, -0.03642227, -1.8372649 , -1.254374  , -1.2776358 ,
        -0.23177852], dtype=float32),
 Array([-0.2456366 ,  0.3126293 ,  1.0859151 ,  0.05576393, -0.42242417,
        -1.1261328 ], dtype=float32))

In [4]:
mjdata.qpos[:] = q
mjdata.qvel[:] = v
mjdata.qacc[:] = dv

mujoco.mj_step(mjmodel, mjdata)
mujoco.mj_rnePostConstraint(mjmodel, mjdata)

velocity = np.zeros(6)
for bodyid in mjmodel.jnt_bodyid:
    mujoco.mj_objectVelocity(mjmodel, mjdata, 2, bodyid, velocity, 1)

    print(f"bodyid: {bodyid}, v: {velocity[3:]}, w: {velocity[:3]}")
    mujoco.mj_objectAcceleration(mjmodel, mjdata, 2, bodyid, velocity, 1)
    print(f"bodyid: {bodyid}, a_lin: {velocity[3:]}, a_ang: {velocity[:3]}")

bodyid: 2, v: [0. 0. 0.], w: [0.         0.         0.95800805]
bodyid: 2, a_lin: [0.   0.   9.81], a_ang: [  0.           0.         133.51350487]
bodyid: 3, v: [0. 0. 0.], w: [ 0.12977856 -0.03642227  0.94917699]
bodyid: 3, a_lin: [1.32893213 0.         9.71956992], a_ang: [ 17.55922354 264.69706853 132.3548755 ]
bodyid: 4, v: [-0.01250669 -0.33221195  0.00246757], w: [ 0.90610408 -1.87368716 -0.31105437]
bodyid: 4, a_lin: [100.0634086  -46.32255204 -21.43708861], a_ang: [126.31661808 137.17039253 -43.24327451]
bodyid: 5, v: [-0.20819569 -0.45166973  0.37383589], w: [ 0.95272099 -3.12806119 -0.10050942]
bodyid: 5, a_lin: [120.10952403 -63.28652317 -24.05620638], a_ang: [ 131.53577472 -955.73404544  -25.7699149 ]
bodyid: 6, v: [ 0.39468402 -0.31301854  0.59280017], w: [ 3.26564387  0.16737446 -1.37814524]
bodyid: 6, a_lin: [88.94263641 95.88811455 42.83847378], a_ang: [ 960.71640941 -134.10758379 9345.2909092 ]
bodyid: 7, v: [ 0.39468402 -0.65554712  0.23874882], w: [ 3.03386535  0.96

In [5]:
with np.printoptions(precision=4, suppress=True):
    print(mjdata.cvel)

[[ 0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.958  -0.1736 -0.1554  0.    ]
 [-0.0279 -0.0234  0.958  -0.1748 -0.154  -0.0089]
 [-1.4343 -1.2056  0.958  -0.2899 -0.017   0.1817]
 [-2.3945 -2.0127  0.958  -0.217  -0.1037  0.4682]
 [-1.577  -2.9853  1.0921 -0.0366  0.0465  0.4585]
 [-1.4011 -2.8451  1.0363 -0.0694  0.0665  0.4055]]


In [6]:
with np.printoptions(precision=4, suppress=True, linewidth=500):
    print(mjdata.cacc)

[[      0.           0.           0.          -0.          -0.           9.81  ]
 [      0.           0.           0.           0.           0.           9.81  ]
 [      0.           0.         133.5135     -24.1929     -21.6563       9.81  ]
 [    202.283      170.7232     133.5135     -15.662      -31.7642      74.1558]
 [    104.9311      88.347      133.5135     -23.6876     -22.2796      87.376 ]
 [   -724.0039    -624.0078     133.5135      38.6625     -94.7821     341.256 ]
 [  -6725.2352    6501.8936    -878.6433   -1286.669    -1201.2067     409.665 ]
 [-129888.0899  -91650.9038   38136.1708   21636.3522  -15194.9754   37568.2828]]


In [11]:
from mujoco_sysid.mjx.regressors import object_velocity, com_acc

mjxdata = mjxdata.replace(qpos=q, qvel=v, qacc=dv)
mjxdata = mjx.step(mjxmodel, mjxdata)
mjxdata = mjx.rne(mjxmodel, mjxdata)

In [12]:
with np.printoptions(precision=4, suppress=True):
    print(mjxdata.cvel)

[[ 0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.958  -0.1736 -0.1554  0.    ]
 [-0.0279 -0.0234  0.958  -0.1748 -0.154  -0.0089]
 [-1.4343 -1.2056  0.958  -0.2899 -0.017   0.1817]
 [-2.3945 -2.0127  0.958  -0.217  -0.1037  0.4682]
 [-1.577  -2.9853  1.0921 -0.0366  0.0465  0.4585]
 [-1.4011 -2.8451  1.0363 -0.0694  0.0665  0.4055]]


In [13]:
with np.printoptions(precision=4, suppress=True, linewidth=500):
    print(com_acc(mjxmodel, mjxdata))

[[  0.       0.       0.       0.       0.       9.81  ]
 [  0.       0.       0.       0.       0.       9.81  ]
 [  0.       0.       0.       0.       0.       9.81  ]
 [ -0.3425   0.4075   0.       0.0204   0.0171   9.814 ]
 [ -0.0728   0.0866   0.      -0.0144  -0.0049   9.8101]
 [  7.6103  -9.054   -0.      -1.1641   3.0386  14.0812]
 [  2.7813 -17.1085 -28.992   -3.4752  -1.0579  11.8998]
 [  1.2405 -29.0586 -63.8849 -11.4691   3.6247  28.2652]]


In [ ]:
for bodyid in mjmodel.jnt_bodyid:
    velocity = object_velocity(mjxmodel, mjxdata, bodyid)

    print(f"bodyid: {bodyid}, v: {velocity[3:]}, w: {velocity[:3]}")

    acceleration = object_acceleration(mjxmodel, mjxdata, bodyid)

    print(f"bodyid: {bodyid}, a_lin: {acceleration[3:]}, a_ang: {acceleration[:3]}")

bodyid: 2, v: [0. 0. 0.], w: [0.         0.         0.95800805]


NameError: name 'object_acceleration' is not defined